# Install requirements

In [1]:
#necessary imports:
import geopandas as gpd
import pandas as pd
import json
import sys
sys.path.append('../')
import copy
import tqdm
import modules.encoder as encoder
import modules.terrain as terrain
import modules.parameters as parameters




ModuleNotFoundError: ignored

# 0. DATASET PREPARATION

# 1. GENERATION
This part can be skipped and if you are not interested in the generation of the file you can directly jump to the section 2. UPDATING and employ the file that is already present in the _import_ folder.

##  1.0 2D GENERATION
### First, we import a geojson that has a series of correctly encoded attributes.

In [ ]:
gdf_small = gpd.read_file("./import/FILLED_DATASET_SUBSET.geojson")

### Then, we convert the geojson into a 2D CityJSON file without any modification.

In [ ]:
cityjson_2d=encoder.gdf_to_CityJSON(gdf_small) # clean and convert the dataframe to a cityjson file (2D!)
#save the 2D cityjson file
with open('./export/2d_cityjson.json', 'w') as outfile:
    json.dump(cityjson_2d, outfile)

## 1.1 TERRAIN GENERATION
### We generate the terrain and translate the footprints to the terrain, obtaining a CityJSON file that contains the terrain and the footprints vertically displaced.

In [ ]:
#if we want to read directly an already generated 2D cityjson file:
with open('./export/2d_cityjson.json') as json_file:
    cityjson_2d = json.load(json_file)

In [ ]:
#create and insert the terrain using opentopodata
cityjson2d_terrain = terrain.insert_terrain_in_cityjson(cityjson_2d, service = 'opentopodata', apikey = None , grid_min_distance = 60, tin_max_error = 1)

In [ ]:
#save the file with terrain
with open('./export/2d_cityjson_with_terrain.json', 'w') as outfile:
    json.dump(cityjson2d_terrain, outfile)

## 1.2 3D GENERATION
### We iterate over each element of the cityjson_2d and we use its attributes to create the 3D cityjson. (LOD1 and LOD2 geometry)

In [ ]:
#if we want to load a cityjson file with terrain:
with open('./export/2d_cityjson_with_terrain.json') as json_file:
    cityjson2d_terrain = json.load(json_file)

In [ ]:
# now we can generate the 3D geometry for each building
cityjson_3d= copy.deepcopy(cityjson2d_terrain)
for id in tqdm.tqdm(cityjson_3d['CityObjects'].keys()): #loop over all the buildings

    cityjson_3d = encoder.update_cityjson_geometry(cityjson_3d, id) #update the cityjson file with the 3D geometry of the building
    if id == 'id-50':
        break

  7%|▋         | 50/767 [01:34<22:40,  1.90s/it] 


In [ ]:
# and save the 3D cityjson file:
with open('./export/3d_cityjson_hip_test.json', 'w') as json_3d:
    json.dump(cityjson_3d, json_3d)

# 2. UPDATING

In [ ]:
#we can import a 3D cityjson file:
latest_cityjson_path = './import/jerusalem_3d.json'
with open(latest_cityjson_path, 'r') as f:
    latest_cityjson = json.load(f)
